In [ ]:
{
    "n": 5,
    "radius": 250.0,
    "n_frames": 45,
    "saveas": "video",
    "workers": null,
    "urls": [
        {
            "url": "mongo/Polygons/latest",
            "prefix": "polygons",
            "query": [
                {
                    "query": {
                        "status_code": 0
                    },
                    "prefix": "polygons/normal"
                },
            ]
        }
    ]
}

In [4]:
import json

print(json.dumps({
    'n': 5,
    'radius': 250.0,
    'n_frames': 45,
    'saveas': 'video',
    'workers': None,
    'urls': [{
        'url': 'mongo/Polygons/latest',
        'prefix': 'polygons', # ignored since query parameter is of type list
        'query': [{
            'query': {'label': 1.0},
            'prefix': 'polygons/flood-probability'
        }, {
            'query': {'waze_flood_status': 1},
            'prefix': 'polygons/waze/flood'
        }, {
            'query': {'lâmina_status': 1},
            'prefix': 'polygons/comando/lâmina'
        }, {
            'query': {'bolsão_status': 1},
            'prefix': 'polygons/comando/bolsão'
        }, {
            'query': {'alagamento_status': 1},
            'prefix': 'polygons/comando/alagamento'
        }, {
            'query': {'alagamento_enchente_status': 1},
            'prefix': 'polygons/comando/alagamento_enchente'
        }, {
            'query': {'enchente_status': 1},
            'prefix': 'polygons/comando/enchente'
        }],
    }],
}, indent=4))

{
    "n": 5,
    "radius": 250.0,
    "n_frames": 45,
    "saveas": "video",
    "workers": null,
    "urls": [
        {
            "url": "mongo/Polygons/latest",
            "prefix": "polygons",
            "query": [
                {
                    "query": {
                        "label": 1.0
                    },
                    "prefix": "polygons/flood-probability"
                },
                {
                    "query": {
                        "waze_flood_status": 1
                    },
                    "prefix": "polygons/waze/flood"
                },
                {
                    "query": {
                        "l\u00e2mina_status": 1
                    },
                    "prefix": "polygons/comando/l\u00e2mina"
                },
                {
                    "query": {
                        "bols\u00e3o_status": 1
                    },
                    "prefix": "polygons/comando/bols\u00e3o"
                },

## Get waze alerts history and open events from bigquery database

In [1]:
cd ../../../../Apps/Python/bolsao-api

C:\Users\luisr\Desktop\Repositories\Apps\Python\bolsao-api


In [30]:
# Python modules
import requests, json, time, pandas as pd, numpy as np, urllib
from datetime import datetime as dt
from google.cloud import bigquery
from google.oauth2 import service_account

# google cloud auth
project_id = 'pluvia-360323'
google_credentials = 'credentials/pluvia-360323-35cd376d5958.json'
# google_credentials = 'credentials/octacity-iduff.json'
# google_credentials = 'credentials/dev-iduff.json'
google_credentials = service_account.Credentials.from_service_account_file(google_credentials)

fields_bq = [
    'city', 'confidence', 'nThumbsUp',
    'street', 'uuid', 'country',
    'type', 'subtype', 'roadType',
    'reliability', 'magvar', 'reportRating',
    'ts', 'geo', 'geoWKT',
]

# Waze partner access token
waze_url = 'https://www.waze.com/partnerhub-api/waze-feed-access-token/c37c11ba-ff9d-4ad5-8ecc-4e4f12e91efb?format=1'

# ---
# Waze partner feed methods

def get_waze_partner_alerts(alert_type='alerts', datetime_as_str=True):
    time_cols = ['pubMillis', 'detectionDateMillis', 'updateDateMillis']
    "alert_type: one of 'alerts', 'irregularities', 'jams'"
    incidents = requests.get(waze_url).json()
    if alert_type not in incidents.keys():
        return None
    df = pd.DataFrame(incidents[alert_type])
    # Data cleaning & preprocessing
    if alert_type == 'alerts':
        df[['longitude', 'latitude']] = list(df['location'].map(lambda coords: [coords['x'], coords['y']]))
        df.drop('location', axis=1, inplace=True)        
    for col in time_cols:
        if col in df.columns:
            df[col] = (df[col] / 1000).map(dt.fromtimestamp)
            df.sort_values(col, ascending=False, inplace=True)
            if datetime_as_str: # Convert datetime column to string
                df[col] = df[col].astype(str)
    df.reset_index(drop=True, inplace=True)
    return df.fillna(np.nan).replace([np.nan], [None])

# ---
# BIGQUERY DATASET METHODS

def bigquery_request(query):
    client = bigquery.Client(credentials=google_credentials)
    query_job = client.query(query)
    return pd.DataFrame(list(map(dict, query_job.result())))

def where_clause(items):
    return ' AND '.join([' '.join([key, 'IN', f"""({', '.join([f'"{value}"' for value in (values if type(values) is list else values.split(','))])})"""]) for key, values in items.items()])

def get_waze_bq_history(
    start=dt(2023,1,1), end='now',
    where={}, select='*', as_datetime=False,
):
    waze_query = '''
    SELECT {} FROM `waze-public-dataset.partner_EscritriodeDadosdoRiodeJaneiro.view_alerts_clustered` WHERE ts BETWEEN "{}" AND "{}"
    '''
    if end == 'now': end = dt.now()
    if type(select) is list: select = ', '.join(select)
    query = waze_query.format(select, str(start), str(end))
    if len(where):
        query = ' '.join([query, 'AND', where_clause(where)])
    history = bigquery_request(query)
    if len(history):
        if not as_datetime:
            for col in ['ts']:
                if col in history.columns:
                    history[col] = history[col].map(lambda stamp: stamp.strftime('%Y-%m-%d %X'))
        return history
    else:
        return pd.DataFrame(columns=fields_bq)

def get_waze_bq_open_events(
    at=dt.now(), where={}, select='*',
    between=7*pd.offsets.Day(), as_datetime=False,
):
    waze_query = '''
        SELECT {} FROM `waze-public-dataset.partner_EscritriodeDadosdoRiodeJaneiro.view_alerts_clustered`
        WHERE ts BETWEEN "{}" AND "{}"
    '''
#         AND data_inicio <= "{}" AND data_fim >= "{}"
    start, end = at - between / 2, at + between / 2
    query = waze_query.format(select, str(at), str(at))
    if len(where): query = ' '.join([query, 'AND', where_clause(where)])
    history = bigquery_request(query)
    if len(history):
        if not as_datetime:
            for col in ['ts']:
                if col in history.columns:
                    history[col] = history[col].map(lambda stamp: stamp.strftime('%Y-%m-%d %X'))
        return history
    else:
        return pd.DataFrame(columns=fields_bq)


In [31]:
data = bigquery_request('SELECT ts FROM `waze-public-dataset.partner_EscritriodeDadosdoRiodeJaneiro.view_alerts_clustered` WHERE ts BETWEEN "2023-02-17 00:00:00" AND "2023-02-18 00:00:00" LIMIT 10')

Forbidden: 403 Access Denied: Table waze-public-dataset:partner_EscritriodeDadosdoRiodeJaneiro.view_alerts_clustered: User does not have permission to query table waze-public-dataset:partner_EscritriodeDadosdoRiodeJaneiro.view_alerts_clustered, or perhaps it does not exist in location US.

Location: US
Job ID: 2af7c42d-b523-4fa9-a78d-39d422fce532


In [25]:
data

,id_pop,id_evento,bairro,data_inicio,data_fim,prazo,descricao,gravidade,latitude,longitude,status,tipo,data_particao
0,1,92731,Barra da tijuca,2023-03-13 06:48:08+00:00,2023-03-13 07:08:17+00:00,Curto,Av. ayrton senna - alt. via parque - barra da ...,Medio,-22.982691,-43.364191,Fechado,Primario,2023-03-13
1,1,92735,Centro,2023-03-13 07:34:01+00:00,2023-03-13 07:35:58+00:00,Curto,Av. alfred agache - sent. aeroporto,Baixo,-22.904908,-43.170419,Fechado,Primario,2023-03-13
2,1,92742,São conrado,2023-03-13 08:42:13+00:00,2023-03-13 09:17:06+00:00,Curto,Estr. eng. fernando mac dowell - alt. fashion ...,Baixo,-22.997270,-43.262898,Fechado,Primario,2023-03-13
3,1,92748,Barra da tijuca,2023-03-13 09:51:51+00:00,2023-03-13 10:02:56+00:00,Curto,Av. das américas - alt. nº 7837 - barra da tijuca,Baixo,-23.000918,-43.389361,Fechado,Primario,2023-03-13
4,1,92753,Santa teresa,2023-03-13 10:46:10+00:00,2023-03-13 10:48:04+00:00,Curto,Túnel rebouças - 2º galeria - sent. lagoa,Baixo,-22.946896,-43.202893,Fechado,Primario,2023-03-13
5,1,92756,Rio comprido,2023-03-13 12:22:42+00:00,2023-03-13 13:44:53+00:00,Curto,Elevado engenheiro freyssinet - praça da bande...,Medio,-22.924621,-43.209588,Fechado,Primario,2023-03-13
6,1,92772,Água santa,2023-03-13 18:42:50+00:00,2023-03-13 19:34:54+00:00,Curto,Linha amarela - alt. saida 2 - sent. fundão,Medio,-22.906995,-43.306705,Fechado,Primario,2023-03-13
7,1,92774,Santa teresa,2023-03-13 19:03:30+00:00,2023-03-13 19:18:42+00:00,Curto,Túnel santa barbara - sent. laranjeiras,Medio,-22.925607,-43.189977,Fechado,Primario,2023-03-13
8,1,92774,Santa teresa,2023-03-13 19:03:30+00:00,2023-03-13 19:18:42+00:00,Curto,Túnel santa barbara - sent. laranjeiras,Medio,-22.925607,-43.189977,Fechado,Primario,2023-03-13
9,1,92776,Praca da bandeira,2023-03-13 19:48:27+00:00,2023-03-13 20:17:52+00:00,Curto,"Elevado eng. freyssinet, alt. rio comprido - s...",Baixo,-22.915335,-43.210110,Fechado,Primario,2023-03-13


In [15]:
data = get_waze_bq_history(
    start=dt(2023,3,10), end='now',
    where={}, select='*', as_datetime=False,
)

Forbidden: 403 Access Denied: Table waze-public-dataset:partner_EscritriodeDadosdoRiodeJaneiro.view_alerts_clustered: User does not have permission to query table waze-public-dataset:partner_EscritriodeDadosdoRiodeJaneiro.view_alerts_clustered, or perhaps it does not exist in location US.

Location: US
Job ID: fb1ab623-57cb-4e12-8f96-a96e1f5b4691


In [ ]:
data